In [1]:

# 필요라이브러리 로딩
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns  # 시각화 라이브러리, matplotlib보다 단순함
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px 
plt.rcParams['font.family'] = 'Malgun Gothic'
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_rows', 200)
import pandas as pd 
import chardet
import sys
import matplotlib
from argparse import Namespace
import os
from tqdm import tqdm
from tqdm.autonotebook import tqdm
tqdm.pandas()

In [2]:
# 로컬 라이브러리 
from hotel import final_hotel_data_processor
from room  import load_room_data
hotel= final_hotel_data_processor()
room= load_room_data()

In [3]:
# 버젼확인 
print("Python 버전:", sys.version)
print("NumPy 버전:", np.__version__)
print("Pandas 버전:", pd.__version__)
print("Matplotlib 버전:", matplotlib.__version__)
print("Seaborn 버전:", sns.__version__)

Python 버전: 3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]
NumPy 버전: 1.26.4
Pandas 버전: 2.2.2
Matplotlib 버전: 3.8.4
Seaborn 버전: 0.12.2


In [4]:
args = Namespace(
    raw_data= './raw_data/',
    output_dir='./raw_data/block/',
    output_price_processed = './raw_data/price_procssed/'
)

# 2년 이상의 시계열 데이터를 가지고 있는 호텔 리스트 로드

## 배경

* 당사의 정책 및 환경상 가격데이터를 꾸준히 적재한 케이스가 드물다
* 본 사업의 목적은 MAPE 10%이하의 고수준의 모델을 만드는것이 목표이므로
* 최대한 데이터 보존율이 높은 세트를 선별해야하는 작업이 필요하다 

## 산출로직
* 호텔별로 월별 평균가를 산출한다
* 적어도 하나의 가격이 있다면 유효값이 도출된다
* 이를 기준으로 2022-01~2023-12-31 까지 24개의 시계열 데이터포인트에 대해서 결측값이 없는 호텔만 모았다

In [14]:
serial=pd.read_excel(args.raw_data+"hotel_list_has_2year_serial.xlsx", index_col=0)

In [15]:
serial

,hotel_id,hotel_name,region,hoteLgrade,업태구분명
0,101,아르반호텔,부산,미등급,숙박업 기타
1,102,솔라리아니시테츠부산,부산,미등급,일반호텔
2,104,부산비즈니스호텔,부산,3성,관광호텔업
3,105,스탠포드인부산,부산,3성,관광호텔업
4,106,크라운하버호텔부산,부산,미등급,일반호텔
...,...,...,...,...,...
1334,1669,브라운도트호텔울산삼산,울산,1성,관광호텔업
1339,1674,켄싱턴리조트서귀포,제주,미등급,휴양콘도미니엄업
1341,1676,소노벨비발디파크bc구오크파인,강원,미등급,미분류
1343,1678,휘닉스평창호텔,강원,4성,관광호텔업


In [16]:
# 메타정보 붙이고 타겟테이블형성
target=hotel[hotel['hotel_id'].isin(serial['hotel_id'])]
target

,hotel_name,road_addr,addr,hotel_id,regino_sigugun,region,lat,lng,hoteLgrade,객실수,호텔규모,업태구분명,market
0,아르반호텔,부산 부산진구 중앙대로691번길 32,부산 부산진구 부전동 517-60,101,부산진구,부산,35,129,미등급,94,100개 미만,숙박업 기타,국내
1,솔라리아니시테츠부산,부산 부산진구 서면로 20,부산 부산진구 부전동 233-16,102,부산진구,부산,35,129,미등급,203,200개 이상,일반호텔,국내
2,부산비즈니스호텔,부산 부산진구 부전로 67,부산 부산진구 부전동 512-4,104,부산진구,부산,35,129,3성,233,200개 이상,관광호텔업,국내
3,스탠포드인부산,부산 중구 구덕로 53,부산 중구 남포동5가 57-1,105,중구,부산,35,129,3성,132,200개 미만,관광호텔업,국내
4,크라운하버호텔부산,부산 중구 중앙대로 114,부산 중구 중앙동4가 83-1,106,중구,부산,35,129,미등급,500,200개 이상,일반호텔,국내
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,브라운도트호텔울산삼산,울산 남구 삼산로383번길 6-17,울산 남구 삼산동 216-12,1669,남구,울산,36,129,1성,50,100개 미만,관광호텔업,국내
1477,켄싱턴리조트서귀포,제주특별자치도 서귀포시 이어도로 684,제주특별자치도 서귀포시 강정동 2677,1674,서귀포시,제주,33,126,미등급,246,200개 이상,휴양콘도미니엄업,국내
1479,소노벨비발디파크bc구오크파인,강원 홍천군 서면 한치골길 264,강원 홍천군 서면 팔봉리 1289,1676,홍천군,강원,38,128,미등급,미분류,200개 이상,미분류,국내
1481,휘닉스평창호텔,강원 평창군 봉평면 태기로 174,강원 평창군 봉평면 면온리 1095-1,1678,평창군,강원,38,128,4성,130,200개 미만,관광호텔업,국내


In [20]:
def process_file(file_path, room, hotel):
    df = pd.read_csv(file_path)
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    df = df.rename(columns={'booked_date':'booking_date'})
    
    
    # 메타 정보 리드하기 
    df['booking_date'] = pd.to_datetime(df['booking_date'], errors='coerce')
    df['scanned_date'] = pd.to_datetime(df['scanned_date'], errors='coerce')
    # 월정보 가져오기 (하나만)
    month = df['booking_date'].iloc[0].strftime('%Y-%m')

    print(f"다음은 {month} 가격 데이터 원본에 대한 메타정보입니다.\n\n")
    print("데이터 프레임 레코드 갯수:", format(df.shape[0], ','))
    print("수집된 숙박업체 갯수", format(df['hotel_id'].nunique(), ','))
    print("수집된 객실 갯수:", format(df['room_id'].nunique(), ','))
    print("가장 빠른 날 in booking_date column:", df['booking_date'].min())
    print("가장 빠른 날 in scanned_date column:", df['scanned_date'].min())
    print("가장 늦은 날 in booking_date column:", df['booking_date'].max())
    print("가장 늦은 날 in scanned_date column:", df['scanned_date'].max())
    print("수집된 객실 인벤토리 범위:", format(df['remain'].nunique(), ','))
    print("객실가 최저가 ", format(df['price'].min(), ','))
    print("객실가 최고가 ", format(df['price'].max(), ','))
    print("\n ----------------------------------")
    
    # 데이터 가공
    # 중복 수집 호텔 제거 
    df = df[df['hotel_id'].isin(target['hotel_id'])]
    # 날짜 차이 구하기 
    df['date_diff'] = df['booking_date'] - df['scanned_date']
    # 리드타임 30일 이하 만들기
    df = df[df['date_diff'] < pd.Timedelta(days=31)]
    df['scanned_date_date'] = df['scanned_date'].dt.date

    # OTA정보를 위해 룸테이블 로드
 
    room_ota=room[['room_id', 'hotel_id', 'ota_type']]

    # OTA 정보 업데이트
    df=pd.merge(df, room_ota, how='inner',on=['hotel_id','room_id'] )
    
    #국내 주요 OTA만 남기기 
    domestic=['GOODCHOICE', 'YANOLJA', 'AGODA', 'EXPEDIA',  'BOOKING' ]
    # df 테이블 적용
    df=df[df['ota_type'].isin(domestic)]

    # remian 1 보다 크면 전부다 1로 바꾸기
    df.loc[df['remain'] > 1, 'remain'] = 1
    
    if (df['price'] == 0).any():
        df_price = df.query('price == 0').shape[0]
        df_price_p = round((df_price / df.shape[0]) * 100, 0)

        print(f"전체 행갯수는 {format(df.shape[0], ',')}이고 가격이 0인 칼럼의 갯수는 {format(df_price, ',')}이고 이는 {format(df_price_p, ',')}%입니다.")
        print(f"가격 0 버리기 전: {df.shape}")

        df = df.query('price != 0')
        print(f"가격 0 버리기 후: {df.shape}")
    else:
        print("가격이 0인 행이 존재하지 않습니다.")
        
      # 호텔테이블 로드

    # 지역변수 붙이기
    df=pd.merge(df,hotel[['hotel_id', 'region',]], how='inner',on='hotel_id')
    
      # 상하위 1% 이상치 버리기
    try:
        q_low_by_region = df.groupby('region')['price'].quantile(0.01)
        q_hi_by_region = df.groupby('region')['price'].quantile(0.99)

        print("하위 1% 가격 by region:\n", q_low_by_region)
        print("\n상위 1% 가격 by region:\n", q_hi_by_region)

        # 지역별로 상하위 1% 버리기 
        def filter_quantiles(group):
            q_low = group['price'].quantile(0.01)
            q_hi = group['price'].quantile(0.99)
            return group[(group['price'] > q_low) & (group['price'] < q_hi)]

        print("지역별로 상하위 1% 버리기 before:", df.shape)
        df = df.groupby('region').apply(filter_quantiles).reset_index(drop=True)
        print("지역별로 상하위 1% 버리기 after:", df.shape)

    except Exception as e:
        print("An error occurred during quantile calculation and filtering:", str(e))
    

    return df 

In [21]:

# 빈 리스트 만들기
df_stats_list = []

# 베이스 디렉토리 잡아주기
base_dir = 'C:/Users/user/Work/hero_master/price/'

# 서브 디렉터리 루프문 만들기
for root, dirs, files in os.walk(base_dir):
    for directory in dirs:
        
        if directory.startswith(('2022', '2023', '2024')): 
            directory_path = os.path.join(root, directory)
            # 폴더 안에 있는 모든 CSV 파일 넣기
            csv_files = [file for file in os.listdir(directory_path) if file.endswith(".csv")]
            # 반복
            for filename in tqdm(csv_files, desc=f"Processing files in {directory}"):
                try:
                    file_path = os.path.join(directory_path, filename)
                    try:
                        df = process_file(file_path,room, hotel)  # Capture the returned DataFrame
                        df_stats_list.append(df)  # Append the DataFrame to the list
                        print(f'처리완료 {filename} \n\n')
                    except Exception as e:
                        print(f'이슈발생.{filename}: {str(e)}\n\n')
                except Exception as e:
                    print(f'파일 로드 이슈발생.{filename}: {str(e)}\n\n')

# 모든 리스트를 하나로 합쳐서 데이터프레임으로 저장하기
df = pd.concat(df_stats_list, ignore_index=True)

Processing files in 2024_1:  33%|██████████████████▋                                     | 1/3 [00:55<01:50, 55.45s/it]

이슈발생.price_01.csv: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.




Processing files in 2024_1:  67%|█████████████████████████████████████▎                  | 2/3 [00:58<00:24, 24.87s/it]

이슈발생.price_02.csv: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.




Processing files in 2024_1: 100%|████████████████████████████████████████████████████████| 3/3 [00:59<00:00, 19.88s/it]

이슈발생.price_03.csv: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.




ValueError: No objects to concatenate

# 호텔 룸가격데이터 호텔 블록 단위로 나누기

In [34]:
"""# 가격데이터 호텔별로 분산 저장하기 
# 시간 오래 걸리니까 어지간해서는 하지말기 
def append_to_csv(batch, filename):
    # 파일이 없을 경우 헤더와 함께 새 파일을 만들고, 있을 경우 데이터만 추가합니다.
    batch.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename))

# 설정
input_files = [
    f"{args.raw_data}hotel_price_target_2022.csv",
    f"{args.raw_data}hotel_price_target_2023.csv",
    f"{args.raw_data}hotel_price_target_2024_01_03.csv"
]
chunk_size = 50000  # 적절한 청크 사이즈 설정

# 호텔 ID별로 데이터 처리
for file_path in input_files:
    for chunk in tqdm(pd.read_csv(file_path, chunksize=chunk_size)):
        grouped = chunk.groupby('hotel_id')
        for hotel_id, group in grouped:
            output_file = f"{args.output_dir}/hotel_{hotel_id}.csv"
            append_to_csv(group, output_file)"""

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# 호텔 가격 데이터 가공

In [5]:
df=pd.read_csv(args.output_dir+'hotel_139.csv', parse_dates=False)

In [6]:
base_dir = args.output_dir
base_dir

'./raw_data/block/'

In [7]:
csv_files= [file for file in os.listdir(base_dir) if file.endswith(".csv")]
csv_files

['hotel_1007.csv',
 'hotel_1008.csv',
 'hotel_101.csv',
 'hotel_102.csv',
 'hotel_1021.csv',
 'hotel_1022.csv',
 'hotel_1025.csv',
 'hotel_1026.csv',
 'hotel_1028.csv',
 'hotel_104.csv',
 'hotel_105.csv',
 'hotel_106.csv',
 'hotel_1060.csv',
 'hotel_1062.csv',
 'hotel_1066.csv',
 'hotel_108.csv',
 'hotel_1083.csv',
 'hotel_1088.csv',
 'hotel_1090.csv',
 'hotel_1095.csv',
 'hotel_112.csv',
 'hotel_1127.csv',
 'hotel_114.csv',
 'hotel_115.csv',
 'hotel_117.csv',
 'hotel_119.csv',
 'hotel_120.csv',
 'hotel_121.csv',
 'hotel_122.csv',
 'hotel_123.csv',
 'hotel_1232.csv',
 'hotel_1235.csv',
 'hotel_124.csv',
 'hotel_1272.csv',
 'hotel_1284.csv',
 'hotel_1294.csv',
 'hotel_1295.csv',
 'hotel_1296.csv',
 'hotel_1297.csv',
 'hotel_1309.csv',
 'hotel_1310.csv',
 'hotel_1311.csv',
 'hotel_1312.csv',
 'hotel_1316.csv',
 'hotel_1336.csv',
 'hotel_1337.csv',
 'hotel_1344.csv',
 'hotel_1346.csv',
 'hotel_1348.csv',
 'hotel_1350.csv',
 'hotel_1357.csv',
 'hotel_1358.csv',
 'hotel_1365.csv',
 'hotel_1

In [131]:
# 전체루프 돌리기 
tqdm.pandas()

for file in tqdm(csv_files):

    file_path = os.path.join(base_dir, file)
    
    # 호텔별 인스턴스 가져오기 
    df = pd.read_csv(file_path)
    
    # 날짜 데이터 형식변환
    converted_df=process_dates_and_sample(df)
    
    # 리드타임 칼럼 형성 
    converted_df_with_lead = create_lead_time_column(converted_df)
    
    # 종가 데이터 추출 
    filtered_df = Create_end_time_df(converted_df_with_lead)
    
    # 메타 데이터확인 및 인스턴스 생성 
    instance_list=create_instance_list(filtered_df)
    
    # 시계열기간 추출 
    total_days,ground_data_count,start_date, end_date,all_dates=data_range_calculator(filtered_df)
    
    # 유효한 인스턴스 빈 리스트 만들기
    validate_instance_list =[]
    
    # 데이터 보유수준이 기간대비 70% 이상 보유하고 있는 최소 인스턴스 확인
    validated_instance_list, validated_instance_y= data_count_processor(instance_list,filtered_df,ground_data_count)
    
    # 결측 북킹데이트 마킹 
    validated_instance_list_updated, validated_instance_df=get_missing_date(validated_instance_list, filtered_df,all_dates)
    
    # # 북킹 데이트별 missing_date 확인하고 연속으로 missing일대 7일 이하인 인스턴스만 남기기
    validate_df, validated_conse_instance_df=missing_check_list_df(validated_instance_df)
    
    # 처리 데이터 아웃풋
    save_hotel_data(validate_df,args.output_price_processed)
    
    # print
    print('\n')

  0%|          | 0/220 [00:00<?, ?it/s]

room_id
 209761104   2024-03-30
 276328103   2024-02-08
-90904037    2022-04-09
 214462679   2023-12-13
-5696732     2022-04-20
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    287
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 320
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'BOOKING' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 11
유효 인스턴스 갯수비율: 3.4375000000000004


room_id
 197691187   2023-09-21
 6626949     2022-11-04
 276335021   2024-03-31
 255921011   2023-11-30
-6047393     2022-01-13
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    82
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 636
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['AGODA' 'GOODCHOICE' 'BOOKING' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01


데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 602
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'EXPEDIA' 'BOOKING' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 15
유효 인스턴스 갯수비율: 2.4916943521594686


room_id
23490       2022-10-21
259900625   2023-12-27
235315101   2023-11-19
38087123    2023-10-03
230221304   2023-12-27
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    618
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 290
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'BOOKING' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 19
유효 인스턴스 갯수비율: 6.551724137931035


room_id
257549223   2024-02-24
47097143    2023-08-26
244504514   2023-11-25
106032821   2023-09-09
47099288    2023-09-30
Name: booking_date

유효 데이터갯수가 넘은 인스턴스 갯수: 18
유효 인스턴스 갯수비율: 2.8938906752411575


room_id
209559827   2023-10-31
223442585   2024-01-29
4506        2022-09-01
224242380   2024-03-14
7005        2023-12-11
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    483
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 1037
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'BOOKING' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 28732158    2022-12-30
 136895253   2023-07-02
-6069234     2022-08-24
-6188478     2022-08-24
 182691288   2023-09-06
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    1131
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 318
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'BOOKING' 

다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    10
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 200
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['EXPEDIA' 'GOODCHOICE' 'BOOKING' 'AGODA' 'YANOLJA']
데이터 시작날짜: 2022-01-02
데이터 종료날짜: 2024-01-18
날짜 범위의 길이 : 747
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 23904
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 208443593   2024-01-19
 6612555     2023-11-16
-6062594     2022-08-23
 222586330   2023-11-13
 41728297    2023-02-28
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    147
-2 days +20:00:00      3
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 333
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['EXPEDIA' 'AGODA' 'GOODCHOICE' 'BOOKING' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 11
유효 인스턴스 갯수비율: 3.303303303303303


room_id
-5763495    2022-03

...

데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 98
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['AGODA' 'EXPEDIA' 'GOODCHOICE' 'BOOKING' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 4
유효 인스턴스 갯수비율: 4.081632653061225


room_id
 103976402   2024-03-29
 131950097   2023-05-31
-9635071     2022-08-23
 116495814   2024-03-14
 1039635     2023-05-12
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 Series([], Name: count, dtype: int64)
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 158
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'EXPEDIA' 'BOOKING' 'AGODA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 30
해당범위 예약일별 필요시계열 데이터갯수: 25451
유효 데이터갯수가 넘은 인스턴스 갯수: 3
유효 인스턴스 갯수비율: 1.89873417721519


room_id
 24435862   2024-03-29
-5694964    2023-05-02
 49651793   2023-07-08
 6528       2024-03-31
 27296373   2022-12-25
Name: booking_date, dtype: datetime64[ns]

유효 데이터갯수가 넘은 인스턴스 갯수: 10
유효 인스턴스 갯수비율: 3.571428571428571


room_id
39957       2022-07-22
32165       2023-10-10
19840       2023-10-15
19995       2024-03-29
226762410   2023-11-23
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    10
-2 days +23:00:00     4
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 102
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'BOOKING' 'AGODA' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 1061113     2022-09-23
 12230426    2022-11-06
 156373972   2024-01-09
 31955       2022-10-04
-9482430     2022-08-23
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    283
-2 days +23:00:00      1
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 1040
데이터세트 OTA 갯수: 5


다음의 값들은 리드타임 범위에서 벗어났습니다 :
 Series([], Name: count, dtype: int64)
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 98
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['BOOKING' 'AGODA' 'EXPEDIA' 'GOODCHOICE' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2023-12-25
날짜 범위의 길이 : 724
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 23168
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 8054350    2024-01-29
 74251944   2024-03-18
-90917900   2022-08-23
 16750      2024-03-16
 16679      2024-03-16
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    152
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 223
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'BOOKING' 'EXPEDIA' 'YANOLJA' 'AGODA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 15469282   2023-10-03
-6689128    2022-08-23
 31684976   2022-12-29
 94350172   2023-0

다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    9
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 140
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['BOOKING' 'GOODCHOICE' 'AGODA' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-17
날짜 범위의 길이 : 807
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 25824
유효 데이터갯수가 넘은 인스턴스 갯수: 4
유효 인스턴스 갯수비율: 2.857142857142857


room_id
189929364   2024-03-31
256930552   2023-12-31
231771757   2023-12-14
254574365   2024-03-13
256930555   2023-12-31
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    6
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 247
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'BOOKING' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-02
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 820
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26240
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 8069853     2023-12-25
-5670018     2022-03-31
 9300

유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
-91333083   2022-07-28
-91042982   2022-07-31
-92720391   2022-07-28
-5690623    2022-01-29
-5809657    2022-01-29
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    2
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 52
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['GOODCHOICE' 'AGODA' 'EXPEDIA' 'BOOKING' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 30
해당범위 예약일별 필요시계열 데이터갯수: 25451
유효 데이터갯수가 넘은 인스턴스 갯수: 0
유효 인스턴스 갯수비율: 0.0


room_id
 45821423   2023-01-31
 45821425   2024-01-20
-9402739    2022-08-23
-91021680   2022-02-09
 1040338    2023-03-30
Name: booking_date, dtype: datetime64[ns]
리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    2
Name: count, dtype: int64
종가데이터 형성 중 
데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 33
데이터세트 OTA 갯수: 3
데이터세트 OTA 목록: ['GOODCHOICE' 'EXPEDIA' 'YANOLJA']
데이터 시작날짜: 2022-01-01
데이터 종

## 날짜데이터 형식변환

In [9]:
def process_dates_and_sample(df):
    """
    날짜 관련 컬럼을 변환하고, 각 room_id에 대한 최대 booking_date를 샘플링합니다.
    
    Args:
    df (pd.DataFrame): 처리할 데이터 프레임. booking_date, scanned_date,
                       scanned_date_date, date_diff 컬럼을 포함해야 합니다.
                       
    Returns:
    pd.Series: 각 room_id 별로 최대 booking_date의 샘플.
    """
    print("날짜데이터 형식변환 중")
    # 날짜 형식 변환
    df['booking_date'] = pd.to_datetime(df['booking_date'], yearfirst=True)
    df['scanned_date'] = pd.to_datetime(df['scanned_date'], yearfirst=True)
    df['scanned_date_date'] = pd.to_datetime(df['scanned_date_date'], yearfirst=True)
    
    # 기간 형식 변환
    df['date_diff'] = pd.to_timedelta(df['date_diff'])
    
    # 시간 추출
    df['scanned_date_time'] = df['scanned_date'].dt.time
    
    # 각 room_id에 대한 최대 booking_date 구하고 샘플링
    print(df.groupby("room_id")['booking_date'].max().sample(5))
    return df

In [7]:
df

,hotel_id,room_id,booking_date,scanned_date,remain,price,date_diff,scanned_date_date,ota_type,region
0,139,-5649934,2022-01-01,2022-01-01 01:00:00,1,170000,-1 days +23:00:00,2022-01-01,AGODA,부산
1,139,-5649934,2022-01-01,2022-01-01 19:00:00,1,190000,-1 days +05:00:00,2022-01-01,AGODA,부산
2,139,-5649934,2022-01-01,2022-01-01 23:00:00,1,190000,-1 days +01:00:00,2022-01-01,AGODA,부산
3,139,-5649305,2022-01-01,2022-01-01 01:00:00,1,280000,-1 days +23:00:00,2022-01-01,AGODA,부산
4,139,-5649305,2022-01-01,2022-01-01 23:00:00,1,280000,-1 days +01:00:00,2022-01-01,AGODA,부산
...,...,...,...,...,...,...,...,...,...,...
3280164,139,292584745,2024-03-31,2024-03-27 15:00:00,0,229900,3 days 09:00:00,2024-03-27,AGODA,부산
3280165,139,292584745,2024-03-31,2024-03-27 16:00:00,1,229900,3 days 08:00:00,2024-03-27,AGODA,부산
3280166,139,292584745,2024-03-31,2024-03-27 23:00:00,1,229900,3 days 01:00:00,2024-03-27,AGODA,부산
3280167,139,292584745,2024-03-31,2024-03-28 00:00:00,1,229900,3 days 00:00:00,2024-03-28,AGODA,부산


In [10]:
converted_df=process_dates_and_sample(df)

날짜데이터 형식변환 중
room_id
291486327   2024-03-23
218262446   2023-10-26
31422       2022-08-13
41932       2024-03-31
8601        2024-03-16
Name: booking_date, dtype: datetime64[ns]


## 리드타임 수정
* 이전 전처리 작업은 리드타임 당일부터 30일을 기준으로 만들어 낸 자료이다
* 데이터를 확인한 하니 31일(**30-31days**), 예약일+1**(-2days)** 도 존재하였다
* -1~31까지 33개의 조건과 선택지가 필요하다

In [11]:
def create_lead_time_column(df):
    """
    date_diff 열에 기반하여 lead_time 열을 생성하는 함수.
    
    Args:
    df (pd.DataFrame): 처리할 데이터 프레임. date_diff 컬럼을 포함해야 함.
    
    Returns:
    pd.DataFrame: lead_time 열이 추가된 데이터 프레임.
    """
    print("리드타임 칼럼 형성 중 ")
    # 조건과 선택지 생성
    conditions = [(df['date_diff'].dt.days >= i) & (df['date_diff'].dt.days < i + 1) for i in range(-1, 32)]
    # -1일이상 0일미만 == 0 
    # 0일이상 1인 마만 == 1
    #  ..
    # 30일 이상 31일 == 30.
    choices = [i for i in range(33)]
    
    # lead_time 열 생성
    df['lead_time'] = np.select(conditions, choices, default=np.nan)
    # 널값 행 필터 
    lead_na=df[df['lead_time'].isnull()]
    print("다음의 값들은 리드타임 범위에서 벗어났습니다 :\n",lead_na.date_diff.value_counts())
    
    return df

In [12]:
converted_df_with_lead = create_lead_time_column(converted_df)

리드타임 칼럼 형성 중 
다음의 값들은 리드타임 범위에서 벗어났습니다 :
 date_diff
-2 days +01:00:00    630
Name: count, dtype: int64


##  종가데이터 추출


In [13]:
def Create_end_time_df(df):
    # 본함수는 시간이 좀걸림 따로 하는게 좋음
    print("종가데이터 형성 중 ")

    # 그룹화하고 각 그룹별로 scanned_date의 가장 마지막 값을 선택
    result_df = df.groupby(['hotel_id','room_id', 'ota_type', 'booking_date', 'lead_time'], as_index=False)['scanned_date_time'].max()
    # 필터된 행만 inner join으로 남기기
    filtered_df = pd.merge(df, result_df, on=['hotel_id', 'room_id',  'ota_type', 'booking_date', 'lead_time', 'scanned_date_time'])
    filtered_df=pd.merge(filtered_df,hotel[['hotel_id','hotel_name']])
    filtered_df=pd.merge(filtered_df,room[['room_id','room_name']])
    
    # booking_date를 문자열로 변환
    filtered_df['booking_date'] = filtered_df['booking_date'].dt.strftime('%Y-%m-%d')

    return filtered_df

In [14]:
# 종가데이터 추출함수
filtered_df = Create_end_time_df(converted_df_with_lead)
filtered_df

종가데이터 형성 중 


,hotel_id,room_id,booking_date,scanned_date,remain,price,date_diff,scanned_date_date,ota_type,region,scanned_date_time,lead_time,hotel_name,room_name
0,139,-5649934,2022-01-01,2022-01-01 23:00:00,1,190000,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 디럭스 트윈 (AVANI DELUXE Twin)
1,139,-5649305,2022-01-01,2022-01-01 23:00:00,1,280000,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 스위트+미니바 (Avani Suite + Minibar)
2,139,-5647625,2022-01-01,2022-01-01 23:00:00,1,220000,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,[조식 2인] 아바니 디럭스 트윈 ([Breakfast for 2] Avani De...
3,139,-5645948,2022-01-01,2022-01-01 23:00:00,1,295000,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,[조식 3인] 아바니 패밀리 스위트 ([Breakfast for 3] Avani F...
4,139,-5645428,2022-01-01,2022-01-01 23:00:00,1,280000,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,[조식 2인] 아바니 주니어 스위트 ([Breakfast for 2] Avani J...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118560,139,292576837,2024-03-31,2024-03-28 23:00:00,1,229900,2 days 01:00:00,2024-03-28,BOOKING,부산,23:00:00,3,아바니센트럴부산,Special Offer - Junior Suite (Casino voucher &...
1118561,139,292576837,2024-03-31,2024-03-29 23:00:00,0,229900,1 days 01:00:00,2024-03-29,BOOKING,부산,23:00:00,2,아바니센트럴부산,Special Offer - Junior Suite (Casino voucher &...
1118562,139,292584745,2024-03-31,2024-03-26 23:00:00,1,229900,4 days 01:00:00,2024-03-26,AGODA,부산,23:00:00,5,아바니센트럴부산,Special Offer - Junior Suite (Casino voucher &...
1118563,139,292584745,2024-03-31,2024-03-27 23:00:00,1,229900,3 days 01:00:00,2024-03-27,AGODA,부산,23:00:00,4,아바니센트럴부산,Special Offer - Junior Suite (Casino voucher &...


## 인스턴스 생성 및 메타데이터 확인

In [15]:
def create_instance_list(df):
    # 필요한 컬럼만 선택하여 고유한 조합을 포함하는 새 데이터프레임 생성
    selected_columns_df = df[['hotel_id','hotel_name', 'room_id', 'room_name', 'ota_type']].drop_duplicates()
    print("데이터세트 고유 호텔 갯수:", df.hotel_id.nunique())
    print("데이터세트 고유 룸 갯수:", df.room_id.nunique())
    print("데이터세트 OTA 갯수:", df.ota_type.nunique())
    print("데이터세트 OTA 목록:", df.ota_type.unique())
    # 선택된 컬럼들만 포함하는 딕셔너리의 리스트 생성
    instance = selected_columns_df.to_dict('records')

    return instance

In [16]:
# 데이터 범위 계산하기 -- 전체 공통
def data_range_calculator(df):
    start_date =  df.booking_date.min()
    end_date=df.booking_date.max()
    print("데이터 시작날짜:", start_date)
    print("데이터 종료날짜:", end_date)
    # 시작 날짜와 종료 날짜 사이의 날짜 범위 생성
    date_range = pd.date_range(start=start_date, end=end_date)
    all_dates = date_range.strftime('%Y-%m-%d').tolist()
    # 날짜 범위의 길이 계산 (총 날짜 수)
    total_days = len(date_range)
    print("날짜 범위의 길이 :", total_days)

    # 리드타임 길이계산
    lead_min=int(df.lead_time.min())
    lead_max=int(df.lead_time.max())
    print("리드타임 최소:", lead_min)
    print("리드타임 최대:", lead_max)
    ground_data_count = total_days * (lead_min+lead_max+1)# 리드타임 0도 실제로 하나인데 0으로 되면 카운트가 안되서 1추가한다
    print("해당범위 예약일별 필요시계열 데이터갯수:", ground_data_count)  # 우리는 시계열 하나당

    return total_days,ground_data_count, start_date, end_date, all_dates

In [17]:
instance_list=create_instance_list(filtered_df)

데이터세트 고유 호텔 갯수: 1
데이터세트 고유 룸 갯수: 354
데이터세트 OTA 갯수: 5
데이터세트 OTA 목록: ['AGODA' 'GOODCHOICE' 'EXPEDIA' 'BOOKING' 'YANOLJA']


In [18]:
total_days,ground_data_count,start_date, end_date,all_dates=data_range_calculator(filtered_df)

데이터 시작날짜: 2022-01-01
데이터 종료날짜: 2024-03-31
날짜 범위의 길이 : 821
리드타임 최소: 0
리드타임 최대: 31
해당범위 예약일별 필요시계열 데이터갯수: 26272


## 인스턴스 별 보유 데이터 건수로 유효성 확인

In [19]:
# 유효한 인스턴스 빈 리스트 만들기
validate_instance_list =[]

In [20]:
#  보유 데이터 건수로 유효 인스턴스 확인
def data_count_validator(df,instance, ground_data_count):

    instance_df=df[df['room_id']==instance['room_id']]
    raw_data_all_count= instance_df.shape[0]
    data_coverage = (raw_data_all_count/ground_data_count)*100

    if data_coverage >= 70:
        instance['validated'] = "Y"
        instance['data_coverage'] = data_coverage

    else :
        instance['validated'] = "N"
        instance['data_coverage'] = data_coverage

    validate_instance_list.append(instance)

    return instance_df

In [21]:
#  data_count_validator 프로세서 및 리스트 만들기
def  data_count_processor(instance_list, df, ground_data_count):
    validate_instance_list = []

    for instance in instance_list:
        data_count_validator(df, instance, ground_data_count)
        validate_instance_list.append(instance)

    validated_instance_list = pd.DataFrame(validate_instance_list)
    validated_instance_y = validated_instance_list.query("validated=='Y'")
    validate_instance_percentage = (validated_instance_y.shape[0] / len(validate_instance_list)) * 100

    print('유효 데이터갯수가 넘은 인스턴스 갯수:', validated_instance_y.shape[0])
    print('유효 인스턴스 갯수비율:', validate_instance_percentage)

    validated_instance_list_dicts = validated_instance_list.to_dict(orient='records')
    validated_instance_y=validated_instance_y.reset_index(drop=True)



    return validated_instance_list_dicts, validated_instance_y

In [22]:
# 데이터 보유수준이 기간대비 70% 이상 보유하고 있는 최소 인스턴스 확인
validated_instance_list, validated_instance_y= data_count_processor(instance_list,filtered_df,ground_data_count)

유효 데이터갯수가 넘은 인스턴스 갯수: 27
유효 인스턴스 갯수비율: 7.627118644067797


In [23]:
validated_instance_y

,hotel_id,hotel_name,room_id,room_name,ota_type,validated,data_coverage
0,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78
1,139,아바니센트럴부산,4066,아바니 프리미어,GOODCHOICE,Y,84
2,139,아바니센트럴부산,4069,아바니 주니어 스위트,GOODCHOICE,Y,87
3,139,아바니센트럴부산,4070,[2인 조식 패키지] 아바니 프리미어,GOODCHOICE,Y,72
4,139,아바니센트럴부산,4071,아바니 패밀리 스위트,GOODCHOICE,Y,84
5,139,아바니센트럴부산,4073,[2인 조식 패키지] 아바니 주니어 스위트,GOODCHOICE,Y,75
6,139,아바니센트럴부산,4074,아바니 스위트,GOODCHOICE,Y,88
7,139,아바니센트럴부산,4075,[3인 조식 패키지] 아바니 패밀리 스위트,GOODCHOICE,Y,71
8,139,아바니센트럴부산,4077,[S컬렉션] 아바니 스위트 플러스,GOODCHOICE,Y,86
9,139,아바니센트럴부산,5192,프리미어룸 (Avani),EXPEDIA,Y,71


## 누락 날짜 확인 및 마킹

In [24]:
# 북킹 데이트 기준 결측 날짜 확인
def get_missing_date(validated_instance_list, df,all_dates):

    updated_instances = []


    for instance in validated_instance_list:
        if instance['validated']== "Y":
            instance_df=df[df['room_id']==instance['room_id']]
            instance_all_booking_date = instance_df['booking_date'].unique().tolist()


            # 누락된 날짜 찾기
            missing_booking_dates = [date for date in all_dates if date not in instance_all_booking_date]

            instance['missing_date']=missing_booking_dates


        else :
            hotel_name = instance['hotel_name']
            room_name = instance['room_name']
            ota_type = instance['ota_type']
            #print(f'{hotel_name}_{room_name}_{ota_type} 유효하지 않습니다')
            instance['missing_date'] = 'X'


        updated_instances.append(instance)
        validated_instance_df=pd.DataFrame(updated_instances).query('validated=="Y"')


    return updated_instances, validated_instance_df

In [25]:
# 결측 북킹데이트 마킹 
validated_instance_list_updated, validated_instance_df=get_missing_date(validated_instance_list, filtered_df,all_dates)

In [26]:
validated_instance_df

,hotel_id,hotel_name,room_id,room_name,ota_type,validated,data_coverage,missing_date
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,"[2022-10-20, 2022-12-03, 2022-12-17, 2022-12-2..."
9,139,아바니센트럴부산,4066,아바니 프리미어,GOODCHOICE,Y,84,"[2022-10-17, 2022-10-18, 2022-10-19, 2022-10-2..."
12,139,아바니센트럴부산,4069,아바니 주니어 스위트,GOODCHOICE,Y,87,"[2023-03-25, 2023-10-07, 2023-10-08, 2023-10-2..."
13,139,아바니센트럴부산,4070,[2인 조식 패키지] 아바니 프리미어,GOODCHOICE,Y,72,"[2022-05-16, 2022-05-17, 2022-05-18, 2022-05-1..."
14,139,아바니센트럴부산,4071,아바니 패밀리 스위트,GOODCHOICE,Y,84,"[2022-08-13, 2022-10-14, 2022-10-15, 2022-12-1..."
15,139,아바니센트럴부산,4073,[2인 조식 패키지] 아바니 주니어 스위트,GOODCHOICE,Y,75,"[2022-05-16, 2022-05-17, 2022-05-18, 2022-05-1..."
16,139,아바니센트럴부산,4074,아바니 스위트,GOODCHOICE,Y,88,"[2022-10-14, 2022-10-15, 2022-12-24, 2023-03-2..."
17,139,아바니센트럴부산,4075,[3인 조식 패키지] 아바니 패밀리 스위트,GOODCHOICE,Y,71,"[2022-05-16, 2022-05-17, 2022-05-18, 2022-05-1..."
18,139,아바니센트럴부산,4077,[S컬렉션] 아바니 스위트 플러스,GOODCHOICE,Y,86,"[2022-07-22, 2022-07-23, 2022-10-01, 2022-10-1..."
23,139,아바니센트럴부산,5192,프리미어룸 (Avani),EXPEDIA,Y,71,"[2023-10-18, 2023-10-19, 2023-10-21, 2023-10-2..."


In [27]:
# 북킹데이트를 기준으로 연속으로 7일 이상 없는 데이터값 식별하기  1 

def explode_dates_and_combine_names(df, date_col='missing_date', name_cols=('hotel_name', 'room_name')):
    """
    주어진 데이터프레임에서 날짜를 분리하고, 호텔과 방 이름을 결합합니다.
    
    Args:
    df (DataFrame): 처리할 데이터 프레임.
    date_col (str): 분리할 날짜 컬럼 이름.
    name_cols (tuple): 결합할 이름 컬럼들.
    
    Returns:
    DataFrame: 처리된 데이터 프레임.
    """
    df_exploded = df.explode(date_col)
    df_exploded['identifier'] = df_exploded[name_cols[0]] + "_" + df_exploded[name_cols[1]]
    df_exploded[date_col] = pd.to_datetime(df_exploded[date_col])
    df_exploded['missing_date'] = pd.to_datetime(df_exploded['missing_date'])
    
    return df_exploded


In [28]:
# identifier 별로 그룹화하고, 각 그룹 내에서 연속된 날짜의 최대 길이를 계산하는 함수를 적용
def check_consecutive_dates(group):
    # missing_date를 정렬
    sorted_dates = group.sort_values()

    # 연속된 날짜의 길이 계산
    deltas = sorted_dates.diff().dt.days == 1
    consecutive_lengths = deltas.groupby((deltas == False).cumsum()).cumsum()

    # 7일 이상 연속되는 경우가 있는지 확인
    return 'N' if any(consecutive_lengths >= 7) else 'Y'


In [29]:
# 북킹 데이트별 missing_date 확인하고 연속으로 missing일대 7일 이하인 인스턴스만 남기기
def missing_check_list_df(df):
    
    # missing_date 행단위로 펼치기 
    df_exploded = explode_dates_and_combine_names(df)
    
    # missing_date 7일 이상 연속되는 경우가 있는지 확인하기 
    df_exploded['consecutive'] = df_exploded.groupby('identifier')['missing_date'].transform(check_consecutive_dates)
    
    # 원래 데이터 프레임에 붙혀서 식별하기
    validated_conse_instance_df= pd.merge(validated_instance_df,df_exploded[['room_id','consecutive']].drop_duplicates(), how='left', on='room_id')
    validated_conse_instance_df=validated_conse_instance_df.query('consecutive=="Y"')
    
    # 유효 인스터스만 남기기
    validate_df=filtered_df[filtered_df['room_id'].isin(validated_conse_instance_df['room_id'].unique())]
    validate_df=validate_df[['hotel_id','hotel_name','ota_type','room_id','room_name','booking_date','lead_time','remain','price']]
    
    return validate_df, validated_conse_instance_df
    

In [36]:
# missing_date 행단위로 펼치기 
df_exploded = explode_dates_and_combine_names(validated_instance_df)
    # missing_date 7일 이상 연속되는 경우가 있는지 확인하기 
df_exploded['consecutive'] = df_exploded.groupby('identifier')['missing_date'].transform(check_consecutive_dates)
    
    # 원래 데이터 프레임에 붙혀서 식별하기
validated_conse_instance_df= pd.merge(validated_instance_df,df_exploded[['room_id','consecutive']].drop_duplicates(), how='left', on='room_id')
validated_conse_instance_df=validated_conse_instance_df.query('consecutive=="Y"')

In [40]:
validate_df=filtered_df[filtered_df['room_id'].isin(validated_conse_instance_df['room_id'].unique())]
validate_df=validate_df[['hotel_id','hotel_name','ota_type','room_id','room_name','booking_date','lead_time','remain','price']]

In [41]:
validate_df

,hotel_id,hotel_name,ota_type,room_id,room_name,booking_date,lead_time,remain,price
26,139,아바니센트럴부산,AGODA,6015,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,2022-01-01,0,1,169828
27,139,아바니센트럴부산,AGODA,6016,아바니 디럭스 시티뷰 트윈룸 (Avani Deluxe City View Twin R...,2022-01-01,0,1,169828
28,139,아바니센트럴부산,AGODA,6018,아바니 디럭스 마운틴뷰 킹룸 (Avani Deluxe Mountain View Ki...,2022-01-01,0,1,178766
29,139,아바니센트럴부산,AGODA,6019,아바니 디럭스 시티뷰 킹룸 (Avani Deluxe City View King Room),2022-01-01,0,1,188055
31,139,아바니센트럴부산,AGODA,6022,아바니 주니어룸 (Avani Junior Room),2022-01-01,0,1,223875
...,...,...,...,...,...,...,...,...,...
1118501,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,4,1,187000
1118502,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,3,1,187000
1118503,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,2,1,187000
1118504,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,1,1,187000


In [43]:
# 필요한것만 남기기
df_exploded=df_exploded[['hotel_id','hotel_name','room_id','room_name','ota_type','missing_date']]
df_exploded

,hotel_id,hotel_name,room_id,room_name,ota_type,missing_date
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-10-20
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-03
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-17
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-24
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-30
...,...,...,...,...,...,...
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-27
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-28
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-29
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-30


In [44]:
# remain 칼럼생성 
df_exploded['remain']=1
df_exploded

,hotel_id,hotel_name,room_id,room_name,ota_type,missing_date,remain
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-10-20,1
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-03,1
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-17,1
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-24,1
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,2022-12-30,1
...,...,...,...,...,...,...,...
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-27,1
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-28,1
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-29,1
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,2024-03-30,1


In [104]:
validate_df, validated_conse_instance_df=missing_check_list_df(validated_instance_df)
validate_df

,hotel_id,hotel_name,ota_type,room_id,room_name,booking_date,lead_time,remain,price
26,139,아바니센트럴부산,AGODA,6015,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,2022-01-01,0,1,169828
27,139,아바니센트럴부산,AGODA,6016,아바니 디럭스 시티뷰 트윈룸 (Avani Deluxe City View Twin R...,2022-01-01,0,1,169828
28,139,아바니센트럴부산,AGODA,6018,아바니 디럭스 마운틴뷰 킹룸 (Avani Deluxe Mountain View Ki...,2022-01-01,0,1,178766
29,139,아바니센트럴부산,AGODA,6019,아바니 디럭스 시티뷰 킹룸 (Avani Deluxe City View King Room),2022-01-01,0,1,188055
31,139,아바니센트럴부산,AGODA,6022,아바니 주니어룸 (Avani Junior Room),2022-01-01,0,1,223875
...,...,...,...,...,...,...,...,...,...
1118501,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,4,1,187000
1118502,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,3,1,187000
1118503,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,2,1,187000
1118504,139,아바니센트럴부산,BOOKING,42074,디럭스 킹룸 - 도시 전망,2024-03-31,1,1,187000


In [98]:
validate_df.columns

Index(['hotel_id', 'room_id', 'booking_date', 'scanned_date', 'remain',
       'price', 'date_diff', 'scanned_date_date', 'ota_type', 'region',
       'scanned_date_time', 'lead_time', 'hotel_name', 'room_name'],
      dtype='object')

In [108]:
hotel_name = validate_df['hotel_name'].unique()[0]
hotel_name

'아바니센트럴부산'

In [111]:
hotel_name = validate_df['hotel_name'].unique()[0]
room_n=validate_df.room_id.nunique()
validate_df.to_csv(args.output_price_processed+f'{hotel_name}_{room_n}.csv', index=False, encoding='utf-8-sig')

14

In [130]:
def save_hotel_data(df, output_path, encoding='utf-8-sig'):
    
    if df.shape[0] == 0:
        pass
    # 호텔 이름을 데이터프레임에서 추출
    else:
        hotel_name = df['hotel_name'].unique()[0]

        # 방 번호의 고유 개수를 계산
        room_n = df['room_id'].nunique()

        # 파일 이름 설정
        file_name = f"{output_path}{hotel_name}_{room_n}.csv"

        # CSV 파일로 저장
        df.to_csv(file_name, index=False, encoding=encoding)

In [121]:
save_hotel_data(validate_df,args.output_price_processed)

In [127]:
df4=pd.DataFrame()
df4

""


In [128]:
df4.shape[0]

0

In [117]:
validate_df.to_csv(args.output_price_processed+f'{hotel_name}_{room_n}.csv', index=False, encoding='utf-8-sig')

In [198]:
# missing_date 행단위로 펼치기 
df_exploded = explode_dates_and_combine_names(validated_instance_df)
# missing_date 7일 이상 연속되는 경우가 있는지 확인하기 
df_exploded['consecutive'] = df_exploded.groupby('identifier')['missing_date'].transform(check_consecutive_dates)
df_exploded

,hotel_id,hotel_name,room_id,room_name,ota_type,validated,data_coverage,missing_date,identifier,consecutive
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,2022-10-20,아바니센트럴부산_아바니 디럭스 트윈,N
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,2022-12-03,아바니센트럴부산_아바니 디럭스 트윈,N
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,2022-12-17,아바니센트럴부산_아바니 디럭스 트윈,N
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,2022-12-24,아바니센트럴부산_아바니 디럭스 트윈,N
7,139,아바니센트럴부산,4062,아바니 디럭스 트윈,GOODCHOICE,Y,78,2022-12-30,아바니센트럴부산_아바니 디럭스 트윈,N
...,...,...,...,...,...,...,...,...,...,...
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,Y,75,2024-03-27,아바니센트럴부산_[2인 조식 패키지] 아바니 스위트,N
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,Y,75,2024-03-28,아바니센트럴부산_[2인 조식 패키지] 아바니 스위트,N
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,Y,75,2024-03-29,아바니센트럴부산_[2인 조식 패키지] 아바니 스위트,N
45,139,아바니센트럴부산,4076,[2인 조식 패키지] 아바니 스위트,GOODCHOICE,Y,75,2024-03-30,아바니센트럴부산_[2인 조식 패키지] 아바니 스위트,N


In [167]:
validated_conse_instance_df= pd.merge(validated_instance_df,df_exploded[['room_id','consecutive']].drop_duplicates(), how='left', on='room_id')
validated_conse_instance_df=validated_conse_instance_df.query('consecutive=="Y"')
validated_conse_instance_df

,hotel_id,hotel_name,room_id,room_name,ota_type,validated,data_coverage,missing_date,consecutive
12,139,아바니센트럴부산,6015,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,Y,88,"[2022-10-24, 2022-10-25, 2022-10-26, 2023-06-2...",Y
13,139,아바니센트럴부산,6016,아바니 디럭스 시티뷰 트윈룸 (Avani Deluxe City View Twin R...,AGODA,Y,89,"[2022-10-24, 2022-10-25, 2022-10-26]",Y
14,139,아바니센트럴부산,6018,아바니 디럭스 마운틴뷰 킹룸 (Avani Deluxe Mountain View Ki...,AGODA,Y,81,"[2022-06-27, 2022-06-28, 2022-06-29, 2022-06-3...",Y
15,139,아바니센트럴부산,6019,아바니 디럭스 시티뷰 킹룸 (Avani Deluxe City View King Room),AGODA,Y,78,"[2022-06-27, 2022-06-28, 2022-06-29, 2022-06-3...",Y
16,139,아바니센트럴부산,6022,아바니 주니어룸 (Avani Junior Room),AGODA,Y,94,[],Y
17,139,아바니센트럴부산,6023,아바니 스위트 (Avani Suite),AGODA,Y,94,[],Y
18,139,아바니센트럴부산,6636,아바니 프리미어룸 (Avani Premier Room),AGODA,Y,78,[2022-07-10],Y
19,139,아바니센트럴부산,41928,디럭스 트윈룸 - 산 전망,BOOKING,Y,87,"[2022-10-24, 2022-10-25, 2022-10-26, 2023-06-1...",Y
20,139,아바니센트럴부산,41929,디럭스 트윈룸 - 도시 전망,BOOKING,Y,88,"[2022-10-24, 2022-10-25, 2022-10-26, 2023-06-12]",Y
21,139,아바니센트럴부산,41930,프리미어 더블룸,BOOKING,Y,92,[],Y


In [260]:
# 유효 인스터스만 남기기
validate_df=filtered_df[filtered_df['room_id'].isin(validated_conse_instance_df['room_id'].unique())]
validate_df

,hotel_id,room_id,booking_date,scanned_date,remain,price,date_diff,scanned_date_date,ota_type,region,scanned_date_time,lead_time,hotel_name,room_name
26,139,6015,2022-01-01,2022-01-01 23:00:00,1,169828,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...
27,139,6016,2022-01-01,2022-01-01 23:00:00,1,169828,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 디럭스 시티뷰 트윈룸 (Avani Deluxe City View Twin R...
28,139,6018,2022-01-01,2022-01-01 23:00:00,1,178766,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 디럭스 마운틴뷰 킹룸 (Avani Deluxe Mountain View Ki...
29,139,6019,2022-01-01,2022-01-01 23:00:00,1,188055,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 디럭스 시티뷰 킹룸 (Avani Deluxe City View King Room)
31,139,6022,2022-01-01,2022-01-01 23:00:00,1,223875,-1 days +01:00:00,2022-01-01,AGODA,부산,23:00:00,0,아바니센트럴부산,아바니 주니어룸 (Avani Junior Room)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118501,139,42074,2024-03-31,2024-03-27 23:00:00,1,187000,3 days 01:00:00,2024-03-27,BOOKING,부산,23:00:00,4,아바니센트럴부산,디럭스 킹룸 - 도시 전망
1118502,139,42074,2024-03-31,2024-03-28 23:00:00,1,187000,2 days 01:00:00,2024-03-28,BOOKING,부산,23:00:00,3,아바니센트럴부산,디럭스 킹룸 - 도시 전망
1118503,139,42074,2024-03-31,2024-03-29 23:00:00,1,187000,1 days 01:00:00,2024-03-29,BOOKING,부산,23:00:00,2,아바니센트럴부산,디럭스 킹룸 - 도시 전망
1118504,139,42074,2024-03-31,2024-03-30 23:00:00,1,187000,0 days 01:00:00,2024-03-30,BOOKING,부산,23:00:00,1,아바니센트럴부산,디럭스 킹룸 - 도시 전망


## 결측값 보강

* 북킹데이트 기준으로 누락값이 없는 인스턴스 먼저 보강을 한다
* 이 경우는 특정 북킹날짜에 적어도 하나의 리드타임에는 가격을 보유한 경우를 말한다
* 예를들어 2023-01-02 , 숙박 예정일이였고 , 이 데이터를 확인했을때를 기준으로 D-1, D-2 ...D-29,D-30  등 30개의 데이터가 존재하는데 
* 이중에서 단 하나라도 있는것들 위주로 이 값을 기준으로  결측값을 보강한뒤
* 이후에 하나도 없는것은 back fill로 보강한다 전날또는 그다음날 데이터

In [169]:
# 공휴일 데이터 로드
holiday=pd.read_csv(args.raw_data+"/holiday_calender_2022_2024.csv")
holiday

,date,day_of_week,dateName,high_demand
0,2022-01-01,Saturday,1월1일,Y
1,2022-01-02,Sunday,N,N
2,2022-01-03,Monday,N,N
3,2022-01-04,Tuesday,N,N
4,2022-01-05,Wednesday,N,N
...,...,...,...,...
1091,2024-12-27,Friday,N,Y
1092,2024-12-28,Saturday,N,Y
1093,2024-12-29,Sunday,N,N
1094,2024-12-30,Monday,N,N


In [261]:
# 데이터 결합 
validate_df=pd.merge(validate_df,holiday, how='inner', left_on='booking_date', right_on='date')
validate_df=validate_df[['room_id','hotel_name','hotel_id','room_name','ota_type','booking_date','lead_time','scanned_date_time','high_demand','dateName','day_of_week','remain','price']]
validate_df

,room_id,hotel_name,hotel_id,room_name,ota_type,booking_date,lead_time,scanned_date_time,high_demand,dateName,day_of_week,remain,price
0,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,169828
1,6016,아바니센트럴부산,139,아바니 디럭스 시티뷰 트윈룸 (Avani Deluxe City View Twin R...,AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,169828
2,6018,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 킹룸 (Avani Deluxe Mountain View Ki...,AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,178766
3,6019,아바니센트럴부산,139,아바니 디럭스 시티뷰 킹룸 (Avani Deluxe City View King Room),AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,188055
4,6022,아바니센트럴부산,139,아바니 주니어룸 (Avani Junior Room),AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,223875
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319300,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,4,23:00:00,N,N,Sunday,1,187000
319301,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,3,23:00:00,N,N,Sunday,1,187000
319302,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,2,23:00:00,N,N,Sunday,1,187000
319303,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,1,23:00:00,N,N,Sunday,1,187000


In [275]:
# 리드타임 생성
all_lead_times = np.arange(32)

# 각 그룹 처리를 위한 함수
def ensure_full_lead_times(group):
    existing_lead_times = group['lead_time'].unique()
    missing_lead_times = np.setdiff1d(all_lead_times, existing_lead_times)
    # 누락된 행을 위한 DataFrame 생성
    missing_rows = []
    for lead_time in missing_lead_times:
        missing_rows.append({
            'room_id': group['room_id'].iloc[0],
            'hotel_id': group['hotel_id'].iloc[0],
            'hotel_name': group['hotel_name'].iloc[0],
            'room_name': group['room_name'].iloc[0],
            'ota_type': group['ota_type'].iloc[0],
            'booking_date': group['booking_date'].iloc[0],  # booking_date를 복사
            'lead_time': lead_time,
            'scanned_date_time': "23:00:00",
            'high_demand': group['high_demand'].iloc[0],  # 기존 값 복사
            'dateName': group['dateName'].iloc[0],  # 기존 값 복사
            'day_of_week': group['day_of_week'].iloc[0],  # 기존 값 복사
            'remain': 1,
            'price': 0
        })
    if missing_rows:  # 누락된 행이 있을 경우에만 추가
        group = pd.concat([group, pd.DataFrame(missing_rows)], ignore_index=True)
    return group

# room_id도 포함하여 그룹화 및 함수 적용
result_df = validate_df.groupby(['room_id', 'hotel_name','hotel_id', 'room_name', 'ota_type', 'booking_date']).apply(ensure_full_lead_times).reset_index(drop=True)
result_df

,room_id,hotel_name,hotel_id,room_name,ota_type,booking_date,lead_time,scanned_date_time,high_demand,dateName,day_of_week,remain,price
0,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,169828
1,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,1,23:00:00,Y,1월1일,Saturday,1,0
2,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,2,23:00:00,Y,1월1일,Saturday,1,0
3,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,3,23:00:00,Y,1월1일,Saturday,1,0
4,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,4,23:00:00,Y,1월1일,Saturday,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365440,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,3,23:00:00,N,N,Sunday,1,187000
365441,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,2,23:00:00,N,N,Sunday,1,187000
365442,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,1,23:00:00,N,N,Sunday,1,187000
365443,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,0,23:00:00,N,N,Sunday,1,187000


In [276]:
result_df['booking_date'] = pd.to_datetime(result_df['booking_date'])
result_df

,room_id,hotel_name,hotel_id,room_name,ota_type,booking_date,lead_time,scanned_date_time,high_demand,dateName,day_of_week,remain,price
0,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,0,23:00:00,Y,1월1일,Saturday,1,169828
1,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,1,23:00:00,Y,1월1일,Saturday,1,0
2,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,2,23:00:00,Y,1월1일,Saturday,1,0
3,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,3,23:00:00,Y,1월1일,Saturday,1,0
4,6015,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,2022-01-01,4,23:00:00,Y,1월1일,Saturday,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365440,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,3,23:00:00,N,N,Sunday,1,187000
365441,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,2,23:00:00,N,N,Sunday,1,187000
365442,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,1,23:00:00,N,N,Sunday,1,187000
365443,42074,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,2024-03-31,0,23:00:00,N,N,Sunday,1,187000


In [277]:
# booking_date를 인덱스로 설정하지 않고 다중 인덱스로 처리
result_df.set_index(['room_id', 'booking_date'], inplace=True)

# pivot_table을 사용하여 room_id와 booking_date 별 리드타임에 따른 가격을 칼럼화
# 결측값은 0으로 채움
pivot_df = result_df.pivot_table(index=['room_id', 'booking_date'], columns='lead_time', values='price', fill_value=0)
pivot_df.reset_index(inplace=True)
pivot_df

lead_time,room_id,booking_date,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
0,6015,2022-01-01,169828,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6015,2022-01-02,124246,114455,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6015,2022-01-03,126923,126923,116207,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6015,2022-01-04,124933,122804,123127,114455,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6015,2022-01-05,127161,126049,124683,126923,116207,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415,42074,2024-03-27,0,165000,0,0,0,0,165000,156750,...,149380,126973,149380,149380,149380,149380,126973,149380,124355,146300
11416,42074,2024-03-28,0,0,0,0,0,0,0,181390,...,145112,170720,145112,170720,170720,170720,145112,145112,167200,142120
11417,42074,2024-03-29,220000,0,0,0,0,0,0,0,...,198550,172320,202730,202730,202730,202730,202730,172320,198550,198550
11418,42074,2024-03-30,0,231000,0,0,0,231000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [279]:
result_meta=result_df.drop(columns=['price','lead_time','remain','scanned_date_time'])
result_meta.reset_index(inplace=True)
result_meta=result_meta.drop_duplicates()
result_meta

,room_id,booking_date,hotel_name,hotel_id,room_name,ota_type,high_demand,dateName,day_of_week
0,6015,2022-01-01,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,Y,1월1일,Saturday
32,6015,2022-01-02,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Sunday
64,6015,2022-01-03,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Monday
96,6015,2022-01-04,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Tuesday
128,6015,2022-01-05,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Wednesday
...,...,...,...,...,...,...,...,...,...
365285,42074,2024-03-27,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Wednesday
365317,42074,2024-03-28,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Thursday
365349,42074,2024-03-29,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Friday
365381,42074,2024-03-30,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Saturday


In [280]:
result_final=result_meta.merge(pivot_df, how='left', on=['room_id','booking_date'])
result_final

,room_id,booking_date,hotel_name,hotel_id,room_name,ota_type,high_demand,dateName,day_of_week,0,...,22,23,24,25,26,27,28,29,30,31
0,6015,2022-01-01,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,Y,1월1일,Saturday,169828,...,0,0,0,0,0,0,0,0,0,0
1,6015,2022-01-02,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Sunday,124246,...,0,0,0,0,0,0,0,0,0,0
2,6015,2022-01-03,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Monday,126923,...,0,0,0,0,0,0,0,0,0,0
3,6015,2022-01-04,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Tuesday,124933,...,0,0,0,0,0,0,0,0,0,0
4,6015,2022-01-05,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Wednesday,127161,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415,42074,2024-03-27,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Wednesday,0,...,149380,126973,149380,149380,149380,149380,126973,149380,124355,146300
11416,42074,2024-03-28,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Thursday,0,...,145112,170720,145112,170720,170720,170720,145112,145112,167200,142120
11417,42074,2024-03-29,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Friday,220000,...,198550,172320,202730,202730,202730,202730,202730,172320,198550,198550
11418,42074,2024-03-30,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Saturday,0,...,0,0,0,0,0,0,0,0,0,0


In [281]:
result_final.replace(0, np.nan, inplace=True)
result_final=result_final.drop([30,31], axis=1)
result_final.isnull().sum()

 room_id            0
 booking_date       0
 hotel_name         0
 hotel_id           0
 room_name          0
 ota_type           0
 high_demand        0
 dateName           0
 day_of_week        0
0                1696
1                1700
2                1713
3                1683
4                1638
5                1603
6                1576
7                1562
8                1564
9                1529
10               1514
11               1482
12               1465
13               1477
14               1447
15               1411
16               1381
17               1399
18               1377
19               1380
20               1381
21               1318
22               1294
23               1271
24               1266
25               1269
26               1279
27               1301
28               1291
29               1321
dtype: int64

In [282]:
# 숫자형 열 이름 지정
price_columns = [i for i in range(30)]  # 0부터 15까지

# 가격 정보 열에 대해서만 fillna를 적용
result_final[price_columns] = result_final[price_columns].apply(lambda row: row.fillna(method='bfill').fillna(method='ffill'), axis=1)

result_final

,room_id,booking_date,hotel_name,hotel_id,room_name,ota_type,high_demand,dateName,day_of_week,0,...,20,21,22,23,24,25,26,27,28,29
0,6015,2022-01-01,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,Y,1월1일,Saturday,169828,...,169828,169828,169828,169828,169828,169828,169828,169828,169828,169828
1,6015,2022-01-02,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Sunday,124246,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
2,6015,2022-01-03,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Monday,126923,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
3,6015,2022-01-04,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Tuesday,124933,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
4,6015,2022-01-05,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Wednesday,127161,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415,42074,2024-03-27,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Wednesday,165000,...,126973,126973,149380,126973,149380,149380,149380,149380,126973,149380
11416,42074,2024-03-28,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Thursday,181390,...,145112,145112,145112,170720,145112,170720,170720,170720,145112,145112
11417,42074,2024-03-29,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Friday,220000,...,198550,172320,198550,172320,202730,202730,202730,202730,202730,172320
11418,42074,2024-03-30,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Saturday,231000,...,231000,231000,231000,231000,231000,231000,231000,231000,231000,231000


In [283]:
result_final.columns = ['price_' + str(int(col)) if isinstance(col, float) and col.is_integer() else col for col in result_final.columns]
result_final

,room_id,booking_date,hotel_name,hotel_id,room_name,ota_type,high_demand,dateName,day_of_week,price_0,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
0,6015,2022-01-01,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,Y,1월1일,Saturday,169828,...,169828,169828,169828,169828,169828,169828,169828,169828,169828,169828
1,6015,2022-01-02,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Sunday,124246,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
2,6015,2022-01-03,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Monday,126923,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
3,6015,2022-01-04,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Tuesday,124933,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
4,6015,2022-01-05,아바니센트럴부산,139,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,N,N,Wednesday,127161,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415,42074,2024-03-27,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Wednesday,165000,...,126973,126973,149380,126973,149380,149380,149380,149380,126973,149380
11416,42074,2024-03-28,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,N,N,Thursday,181390,...,145112,145112,145112,170720,145112,170720,170720,170720,145112,145112
11417,42074,2024-03-29,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Friday,220000,...,198550,172320,198550,172320,202730,202730,202730,202730,202730,172320
11418,42074,2024-03-30,아바니센트럴부산,139,디럭스 킹룸 - 도시 전망,BOOKING,Y,N,Saturday,231000,...,231000,231000,231000,231000,231000,231000,231000,231000,231000,231000


In [284]:
result_final=result_final[['room_id','booking_date','hotel_id', 'hotel_name', 'room_name', 'ota_type', 'price_0', 'price_1',
       'price_2', 'price_3', 'price_4', 'price_5', 'price_6', 'price_7',
       'price_8', 'price_9', 'price_10', 'price_11', 'price_12', 'price_13',
       'price_14', 'price_15', 'price_16', 'price_17', 'price_18', 'price_19',
       'price_20', 'price_21', 'price_22', 'price_23', 'price_24', 'price_25',
       'price_26', 'price_27', 'price_28', 'price_29']]
result_final

,room_id,booking_date,hotel_id,hotel_name,room_name,ota_type,price_0,price_1,price_2,price_3,...,price_20,price_21,price_22,price_23,price_24,price_25,price_26,price_27,price_28,price_29
0,6015,2022-01-01,139,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,169828,169828,169828,169828,...,169828,169828,169828,169828,169828,169828,169828,169828,169828,169828
1,6015,2022-01-02,139,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,124246,114455,114455,114455,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
2,6015,2022-01-03,139,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,126923,126923,116207,116207,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
3,6015,2022-01-04,139,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,124933,122804,123127,114455,...,114455,114455,114455,114455,114455,114455,114455,114455,114455,114455
4,6015,2022-01-05,139,아바니센트럴부산,아바니 디럭스 마운틴뷰 트윈룸 (Avani Deluxe Mountain View T...,AGODA,127161,126049,124683,126923,...,116207,116207,116207,116207,116207,116207,116207,116207,116207,116207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11415,42074,2024-03-27,139,아바니센트럴부산,디럭스 킹룸 - 도시 전망,BOOKING,165000,165000,165000,165000,...,126973,126973,149380,126973,149380,149380,149380,149380,126973,149380
11416,42074,2024-03-28,139,아바니센트럴부산,디럭스 킹룸 - 도시 전망,BOOKING,181390,181390,181390,181390,...,145112,145112,145112,170720,145112,170720,170720,170720,145112,145112
11417,42074,2024-03-29,139,아바니센트럴부산,디럭스 킹룸 - 도시 전망,BOOKING,220000,209000,209000,209000,...,198550,172320,198550,172320,202730,202730,202730,202730,202730,172320
11418,42074,2024-03-30,139,아바니센트럴부산,디럭스 킹룸 - 도시 전망,BOOKING,231000,231000,231000,231000,...,231000,231000,231000,231000,231000,231000,231000,231000,231000,231000
